In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from scipy.stats import pearsonr

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
validationData = pd.read_csv('/content/drive/My Drive/Assignment 3 NLP/dev.csv', delimiter="\t")

In [ ]:
null_mask = validationData.isnull().any(axis=1)
validationData[null_mask]

,score,setence1,sentence2
764,1.4,The grass family is one of the most widely dis...,NaN
1024,3.8,"She was crying and scared,' said Isa Yasin, th...",NaN


In [ ]:
validationData.isnull().sum().sum()

2

In [ ]:
validationData

,score,setence1,sentence2
0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,4.75,A young child is riding a horse.,A child is riding a horse.
2,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,2.40,A woman is playing the guitar.,A man is playing guitar.
4,2.75,A woman is playing the flute.,A man is playing a flute.
...,...,...,...
1465,2.00,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?
1466,0.00,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations
1467,2.00,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o..."
1468,0.00,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...


In [ ]:
validationData = validationData.dropna(subset=['setence1', 'sentence2', 'score'])
validationData.reset_index(drop=True, inplace = True)
validationData

,score,setence1,sentence2
0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,4.75,A young child is riding a horse.,A child is riding a horse.
2,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,2.40,A woman is playing the guitar.,A man is playing guitar.
4,2.75,A woman is playing the flute.,A man is playing a flute.
...,...,...,...
1463,2.00,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?
1464,0.00,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations
1465,2.00,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o..."
1466,0.00,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...


In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
cosineScoreList = []
for sen1, sen2 in zip(validationData['setence1'], validationData['sentence2']):
  embeddingSen1 = model.encode(sen1, convert_to_tensor= True)
  embeddingSen2 = model.encode(sen2, convert_to_tensor= True)
  cosineScore = util.cos_sim(embeddingSen1, embeddingSen2)
  cosineScoreList.append(cosineScore[0][0].item())
cosineScoreList

[0.9933716058731079,
 0.9539622664451599,
 0.9801015853881836,
 0.6470829844474792,
 0.7462276220321655,
 0.7546188831329346,
 0.9815390110015869,
 0.6809811592102051,
 0.8914710283279419,
 0.9549217820167542,
 0.865527331829071,
 0.5417836904525757,
 0.9809772968292236,
 0.926334798336029,
 0.9664143323898315,
 0.06189502030611038,
 0.7929203510284424,
 0.956871509552002,
 0.9118955135345459,
 0.36104679107666016,
 0.7287824153900146,
 0.5095634460449219,
 0.7969235181808472,
 0.5152466297149658,
 0.3810451030731201,
 0.5949642658233643,
 0.7490595579147339,
 0.6846779584884644,
 0.34796568751335144,
 -0.03945541754364967,
 0.7508143186569214,
 0.8956352472305298,
 0.9035985469818115,
 0.9790109395980835,
 0.8268659114837646,
 0.9053809642791748,
 0.9832698702812195,
 0.9783763289451599,
 0.9020698070526123,
 0.4285459816455841,
 0.7856063842773438,
 0.7909080982208252,
 0.3393152952194214,
 0.7585870623588562,
 0.5384450554847717,
 0.2569526433944702,
 -0.09588086605072021,
 -0.05738

In [ ]:
updatedLabels = []
for labelVal in cosineScoreList:
  newLabelVal = 4*((labelVal - min(cosineScoreList)) / (max(cosineScoreList) - min(cosineScoreList))) + 1
  updatedLabels.append(newLabelVal)
updatedLabels

[4.976221295338485,
 4.834846752415347,
 4.928617272843593,
 3.733967649777172,
 4.089632779547781,
 4.119735052762456,
 4.9337738015251364,
 3.8555717931749776,
 4.610669682873874,
 4.838288857188754,
 4.5176009258861995,
 3.356223699364713,
 4.9317587440406925,
 4.735737740877791,
 4.879516497123838,
 1.634701786519972,
 4.257135288229803,
 4.845283185008541,
 4.683939174381166,
 2.7078597097746053,
 4.027050961353571,
 3.24063885597164,
 4.271495993956698,
 3.2610263454693937,
 2.7796003747736435,
 3.547000293049581,
 4.099791885150597,
 3.868833454391046,
 2.660933395757743,
 1.271123710001684,
 4.106086801570594,
 4.625608135345567,
 4.6541751673953105,
 4.9247047599138,
 4.378909420029574,
 4.660569297171927,
 4.939982975367538,
 4.922428198258783,
 4.648691063579411,
 2.95000198754128,
 4.230897631841724,
 4.249916661248175,
 2.629901531879484,
 4.133970245040353,
 3.3442468938812455,
 2.3344390241412194,
 1.068706666029032,
 1.2067935103105483,
 4.178266870552075,
 4.6179045603

In [ ]:
coeff, _ = pearsonr(validationData['score'], updatedLabels)

In [ ]:
coeff

0.8631423871595579